In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import urllib.request
import os, pickle, sys, random
import seaborn as sns
from sklearn.manifold import TSNE
from scipy.stats import norm
sns.set(style="whitegrid", palette="rainbow")

In [13]:
from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, DownloadableAnnDataset
from scvi.dataset import BrainLargeDataset, CortexDataset, PbmcDataset, RetinaDataset, HematoDataset, CbmcDataset, BrainSmallDataset, SmfishDataset
save_path = "data/"
from net_prob import GeneGraph

In [3]:
smfish_dataset = SmfishDataset(save_path=save_path)

[2020-01-12 22:28:09,466] INFO - scvi.dataset.dataset | File /Users/sherilynw/Desktop/4_1/计算生物学/singlecell_casuality/data/osmFISH_SScortex_mouse_all_cell.loom already downloaded
[2020-01-12 22:28:09,468] INFO - scvi.dataset.smfish | Loading smFISH dataset
[2020-01-12 22:28:09,723] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2020-01-12 22:28:09,727] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-01-12 22:28:09,729] INFO - scvi.dataset.dataset | Downsampled from 6471 to 4530 cells
[2020-01-12 22:28:09,730] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-01-12 22:28:09,733] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]


In [4]:
smfish_dataset

GeneExpressionDataset object with n_cells x nb_genes = 4530 x 33
    gene_attribute_names: 'gene_names'
    cell_attribute_names: 'y_coord', 'labels', 'local_means', 'local_vars', 'x_coord', 'batch_indices'
    cell_categorical_attribute_names: 'labels', 'batch_indices'

In [8]:
for i in list(smfish_dataset.gene_names):
    print(i)

GAD2
SLC32A1
CRHBP
CNR1
VIP
CPNE5
PTHLH
CRH
TBR1
LAMP5
RORB
SYT6
KCNIP2
ALDOC
GFAP
SERPINF1
MFGE8
SOX10
PLP1
PDGFRA
BMP4
ITPR2
TMEM2
CTPS
ANLN
MRC1
HEXB
TTR
FOXJ1
VTN
FLT1
APLN
ACTA2


In [9]:
len(smfish_dataset.gene_names)

33

In [54]:
string = pd.read_csv("data/string_interactions.tsv", sep="\t")
string = string[string["automated_textmining"]!=string["combined_score"]]
edges = list(zip(string["#node1"], string["node2"]))
edges = [tuple([x[0].upper(), x[1].upper()]) for x in edges]
string

,#node1,node2,node1_string_internal_id,node2_string_internal_id,node1_external_id,node2_external_id,neighborhood_on_chromosome,gene_fusion,phylogenetic_cooccurrence,homology,coexpression,experimentally_determined_interaction,database_annotated,automated_textmining,combined_score
0,Slc32a1,Gad2,5021312,5018993,10090.ENSMUSP00000036299,10090.ENSMUSP00000028123,0,0,0,0,0.472,0.045,0.9,0.931,0.996
1,Crh,Crhbp,5025358,5022387,10090.ENSMUSP00000061185,10090.ENSMUSP00000042578,0,0,0,0,0.064,0.000,0.9,0.956,0.995
2,Crh,Pthlh,5025358,5023861,10090.ENSMUSP00000061185,10090.ENSMUSP00000051433,0,0,0,0,0.083,0.000,0.9,0.190,0.919
3,Bmp4,Mfge8,5027047,5020352,10090.ENSMUSP00000073720,10090.ENSMUSP00000032825,0,0,0,0,0.000,0.000,0.9,0.070,0.903
4,Ttr,Hexb,5027223,5017429,10090.ENSMUSP00000074783,10090.ENSMUSP00000022169,0,0,0,0,0.000,0.000,0.9,0.074,0.903
5,Cnr1,Apln,5024545,5022963,10090.ENSMUSP00000055797,10090.ENSMUSP00000046012,0,0,0,0,0.000,0.000,0.9,0.000,0.900
6,Gfap,Plp1,5025850,5020637,10090.ENSMUSP00000064691,10090.ENSMUSP00000033800,0,0,0,0,0.672,0.000,0.0,0.549,0.846
7,Crh,Cnr1,5025358,5024545,10090.ENSMUSP00000061185,10090.ENSMUSP00000055797,0,0,0,0,0.112,0.000,0.0,0.760,0.778
8,Sox10,Plp1,5021866,5020637,10090.ENSMUSP00000039466,10090.ENSMUSP00000033800,0,0,0,0,0.255,0.000,0.0,0.675,0.748
9,Gfap,Sox10,5025850,5021866,10090.ENSMUSP00000064691,10090.ENSMUSP00000039466,0,0,0,0,0.180,0.000,0.0,0.671,0.719


In [33]:
with open("result_osmFISH_100.pickle", "rb") as f:
    g, _ = pickle.load(file=f)

In [55]:
len(edges), len(g.g.edges)

(36, 55)

In [56]:
scnet = set(list(g.g.edges) + [tuple([x[1].upper(), x[0].upper()]) for x in g.g.edges])
len(scnet)

110

In [57]:
string = set(edges + [tuple([x[1].upper(), x[0].upper()]) for x in edges])
len(string)

72

In [58]:
len([x for x in string if x in scnet])

18

In [59]:
len([x for x in string if x not in scnet])

54

In [60]:
len([x for x in scnet if x not in string])

92

In [61]:
[x for x in string if x not in scnet]

[('CNR1', 'GFAP'),
 ('GAD2', 'VIP'),
 ('SLC32A1', 'GFAP'),
 ('PDGFRA', 'BMP4'),
 ('SLC32A1', 'TBR1'),
 ('GAD2', 'CNR1'),
 ('RORB', 'TBR1'),
 ('SLC32A1', 'VIP'),
 ('MFGE8', 'BMP4'),
 ('GAD2', 'GFAP'),
 ('BMP4', 'MFGE8'),
 ('TBR1', 'SLC32A1'),
 ('PDGFRA', 'SOX10'),
 ('SOX10', 'PDGFRA'),
 ('BMP4', 'PDGFRA'),
 ('TBR1', 'GAD2'),
 ('TBR1', 'RORB'),
 ('FLT1', 'VTN'),
 ('TBR1', 'GFAP'),
 ('HEXB', 'TTR'),
 ('GFAP', 'TBR1'),
 ('FLT1', 'BMP4'),
 ('CNR1', 'APLN'),
 ('CRHBP', 'CRH'),
 ('VTN', 'FLT1'),
 ('CRH', 'CNR1'),
 ('TTR', 'HEXB'),
 ('VIP', 'GAD2'),
 ('GAD2', 'CRH'),
 ('ACTA2', 'PDGFRA'),
 ('GFAP', 'PDGFRA'),
 ('GFAP', 'ALDOC'),
 ('VIP', 'SLC32A1'),
 ('FOXJ1', 'BMP4'),
 ('GAD2', 'TBR1'),
 ('BMP4', 'FLT1'),
 ('CNR1', 'CRH'),
 ('CRH', 'CRHBP'),
 ('GFAP', 'GAD2'),
 ('GFAP', 'SLC32A1'),
 ('PDGFRA', 'ACTA2'),
 ('ALDOC', 'GFAP'),
 ('RORB', 'SYT6'),
 ('BMP4', 'FOXJ1'),
 ('SYT6', 'RORB'),
 ('CRH', 'VIP'),
 ('APLN', 'CNR1'),
 ('CRH', 'GAD2'),
 ('PDGFRA', 'GFAP'),
 ('SOX10', 'PTHLH'),
 ('GFAP', 'CNR1'),